In [1]:
import os
import json
import xnet as xn
import numpy as np
from collections import Counter

In [2]:
path = "../packages/issi_data_package/output_files/data/"
#path = "../networks/"
queryID = "774e7eb6-6ac7-4dd5-9339-531b746cb8ec"
graph = xn.xnet2igraph(path+"/"+queryID+".xnet")

In [3]:
# saving yearly networks

windowSize = 4 #years 
minComponentSize = 20
startYear = 1990 #
endYear = 2018 #

path = "../Visualizer/Data/"

os.makedirs("../networks", exist_ok=True)
os.makedirs("../networks/tmp/", exist_ok=True)

publicationYear = np.array(graph.vs["year"],dtype=np.int)
minYear = max(np.min(publicationYear),startYear)
maxYear = min(np.max(publicationYear),endYear)

firstYear = minYear+windowSize;
lastYear = maxYear;

networksDictionary = {}
for year in range(firstYear,lastYear+1):
    afterWindowStart = publicationYear >= (year-windowSize)
    beforeWindowEnd = publicationYear <= (year)
    selectedVertices = np.where(np.logical_and(afterWindowStart,beforeWindowEnd))[0]
    yearGraph = graph.subgraph(selectedVertices);
    yearGraphCopy = yearGraph.copy()
    yearGraphCopy.to_undirected() #Using Louvain, so the network needs to be undirected
    yearGraph.vs["YearCommunity"] = [str(c) for c in yearGraphCopy.community_multilevel().membership]
    componentMembership = yearGraph.clusters(mode="WEAK").membership;
    removeComponents = set([component for component,freq in Counter(componentMembership).items() if freq<minComponentSize])
    yearGraph.delete_vertices([index for index in range(yearGraph.vcount()) if componentMembership[index] in removeComponents])
    filename = "../networks/tmp/network_w%d_y%d.xnet"%(windowSize,year);
    xn.igraph2xnet(yearGraph,filename);
    with open(filename,"r") as fd:
        networksDictionary[year] = fd.read();
    os.remove(filename);

jupyterData = {
    "firstYear": firstYear,
    "lastYear": lastYear,
    "networks": networksDictionary,
    "displayProperty":"original_title",
    "IDProperty":"paper_id",
    "colorProperty":"YearCommunity",
    "networkname": "network_w%d"%windowSize,
}

with open("./Visualizer/index_template.html","r") as fd:
    indexHTML = fd.read().replace("__JUPYTERDATA__","var JUPYTER_DATA = "+json.dumps(jupyterData))
    
with open("./Visualizer/index.html","w") as fd:
    fd.write(indexHTML)

# with open("./Visualizer/JupyterData.js","w") as fd:
#     fd.write("var JUPYTER_DATA = ")
#     json.dump(jupyterData,fd)
    
from IPython.display import IFrame
IFrame(src='./Visualizer/index.html',width=1000, height=900)